# Automated Network Edge Filter for Junos

Rid yourself of annoying traffic across many systems in a snap.

## The problem? Nuisance traffic.

Got someone beating on your door trying the same attack over and over and over? Someone who thinks it's fun to run nmap scans all day against your network? Dump their traffic with a simple firewall filter at the edge.

## How to make this work
### Step 1: Create 2 prefix-lists on your system.
First is the `edge-block` prefix-list, which is your set of blocked IP addresses or subnets. Second is `edge-block-exceptions`, your list of exceptions to the edge-block prefix-list. An example follows. Please don't use this as-is.

```
user@router> show configuration policy-options
prefix-list edge-block {
    1.1.1.0/24;
    2.2.2.0/24;
    3.3.3.0/24;
}
prefix-list edge-block-exceptions {
    1.1.1.1/32;
    2.2.2.2/32;
    3.3.3.3/32;
}
```

### Step 2: Create a firewall filter.  

It's going to be super simple.  Essentially, it says, "drop everything to & from the blocked list, apart from the exceptions list, and allow all else."  You can (obviously) customize this to your heart's content, but this is the minimum you'll need.
```
user@router> show configuration firewall family inet
filter internet-edge-block {
    term 1 {
        from {
            prefix-list {
                edge-block;
                edge-block-exceptions except;
            }
        }
        then {
            count blocked;
            syslog;
            discard;
        }
    }
    term 2 {
        then accept;
    }
}
```
### Step 3. Apply this filter to your external-facing interface.

Our example assumes you're using ge-0/0/0 as your external-facing interface.
```
user@router> show configuration interfaces ge-0/0/0
description "Peering: ISP1";
unit 0 {
    family inet {
        filter {
            input internet-edge-block;
        }
        address X.Y.Z.2/30;
    }
}
```

### Step 4: Edit the section of YAML in the next block.
It should be pretty self-explanatory how to maintain the info contained therein.

In [ ]:
setup_yaml = """
---
hosts:
    - router1

blocked:
    - 1.2.3.0/24
    - 2.2.2.0/24
    - 3.3.3.0/24

exceptions:
    - 1.2.3.1/32
    - 2.2.2.2/32
    - 3.3.3.3/32
"""

### Step 5: Configure Username & Password

In [ ]:
user = 'autobot'
password = 'juniper123'

### Below here, all the magic happens. Output will show up below next cell.

In [ ]:
from jnpr.junos import Device
from jnpr.junos.utils.config import Config
from jnpr.junos.exception import *
import yaml
from jinja2 import Template
import sys

prefix_lists_template = """
policy-options {
    prefix-list edge-block {
        {% for ip in blocked %}
        {{ ip }};
        {% endfor %}
    }
    prefix-list edge-block-exceptions {
        {% for ip in exceptions %}
        {{ ip }};
        {% endfor %}
    }
}
"""
j2_template = Template(prefix_lists_template)

config = yaml.load(setup_yaml)
hosts = config.get('hosts')

for host in hosts:
    # print("Host: %s" % host)
    dev = Device(host=host, user=user, password=password)

    # Connect to host
    try:
        print("Connecting to {}".format(host))
        dev.open()
        print("Successfully connected to {}".format(host))
    except Exception as err:
        print("Connection to {} failed, reason: {}}".format(host, err))
        sys.exit(0)
    dev.bind(cu=Config)

    # Lock the configuration, load changes, commit
    print("Locking the configuration on: {}".format(host))
    try:
        dev.cu.lock()
    except LockError:
        print("Error: Unable to lock configuration on: {}".format(host))
        dev.close()
        sys.exit(0)

    # Zero out old prefix-lists
    print("Zeroing out old prefix-lists on: {}".format(host))
    try:
        set_commands = """
        delete policy-options prefix-list edge-block
        delete policy-options prefix-list edge-block-exceptions
        """
        dev.cu.load(set_commands, format='set')
        # Load new prefix-lists
        dev.cu.load(template=j2_template,
                    template_vars=config, format='text', merge=True)
    except ValueError as err:
        print(err.message)

    # Commit Configuration
    print("Committing the configuration on: {}".format(host))
    try:
        dev.cu.commit()
        # print dev.cu.diff()
    except CommitError:
        print("Error: Unable to commit configuration on: {}".format(host))
        print("Unlocking the configuration on: {}".format(host))
        try:
            dev.cu.unlock()
        except UnlockError:
            print("Error: Unable to unlock configuration on: {}".format(host))
        dev.close()
        sys.exit(0)

    # Unlock configuration
    print("Unlocking the configuration on: {}".format(host))
    try:
        dev.cu.unlock()
    except UnlockError:
        print ("Error: Unable to unlock configuration on: {}".format(host))
        dev.close()
        sys.exit(0)

    # Disconnect from host
    try:
        print("Closing connection to {}".format(host))
        dev.close()
    except Exception as err:
        print("Disconnect from {} failed, reason: {}".format(host, err))
        sys.exit(0)
